In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough

In [ ]:
model = ChatOllama(
    model = "llama3.2")

In [ ]:
raw_doc = TextLoader("./data.txt").load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300,
chunk_overlap = 20)
document = text_splitter.split_documents(raw_doc)

In [ ]:
oembed = OllamaEmbeddings(model = "nomic-embed-text")

In [ ]:
db = Chroma.from_documents(document,embedding = oembed)

In [ ]:
chat_template_ = """
Answer the question only from the context given below nowhere else
{context}
question: {question}
"""
chat_template = ChatPromptTemplate.from_template(chat_template_)

In [ ]:
retiver = db.as_retriever()

In [ ]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [ ]:
chain = ({"context":retiver|format_docs,"question":RunnablePassthrough()}|chat_template|model)

In [ ]:
chain.invoke("what country is budgie native to").content

'Australia.'

In [ ]:
chain.invoke("what are the color of budgie").content

'The colors commonly associated with budgies (budgerigars) are green, yellow, black, blues, whites, greys.'